In [1]:
import yaml
from jinja2 import Template
from getpass import getpass
from netmiko import Netmiko
from pprint import pprint

In [2]:
def read_inventory(inventory_file):
    with open(inventory_file, 'r') as f:
        inventory = yaml.safe_load(f)
    return inventory

In [3]:
def generate_config_snippet(jinja_template, value):
    with open(jinja_template) as f:
        template = Template(f.read())
        config_snippet = template.render(value)
    return config_snippet

In [4]:
def write_config_snippet(config_snippet, output_file):
    with open(output_file, 'w') as f:
              f.write(config_snippet)

In [5]:
if __name__ == '__main__':
    username=input('Enter your username: ')
    password=getpass('Enter your password: ')

In [6]:
    inventory_file = 'inventory/routers.yml'
    inventory = read_inventory(inventory_file)

In [7]:
    jinja_template = 'templates/eigrp.j2'
    for key, value in inventory.items():
        output_file = f"snippets/{key}_snippet.cfg"
        config_snippet = generate_config_snippet(jinja_template, value)
        write_config_snippet(config_snippet, output_file)
        with Netmiko(host=value['host'], 
                     port=value['port'],
                     username=username, 
                     password=password, 
                     device_type=value['device_type']) as session:
            print(session.find_prompt())
            output = session.send_config_from_file(output_file)
            print(output)

CSR1#
config term
Enter configuration commands, one per line.  End with CNTL/Z.
CSR1(config)#router eigrp 1
CSR1(config-router)#network 1.1.1.1 0.0.0.0
CSR1(config-router)#network 192.168.100.0 0.0.0.255
CSR1(config-router)#end
CSR1#
CSR3#
config term
Enter configuration commands, one per line.  End with CNTL/Z.
CSR3(config)#router eigrp 1
CSR3(config-router)#network 3.3.3.3 0.0.0.0
CSR3(config-router)#network 192.168.100.0 0.0.0.255
CSR3(config-router)#end
CSR3#
